# 2️⃣ Convert PyTorch Model to ONNX

This notebook loads the trained PyTorch model and exports it to ONNX format (`model.onnx`).

In [ ]:
# 1. Mount Google Drive and set root path
from google.colab import drive
drive.mount('/content/drive')
root = '/content/drive/MyDrive/hardware_aware_optimization'

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import os

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(128*8*8, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [ ]:
model = SimpleCNN()
model.load_state_dict(torch.load(f"{root}/models/base_model.pt", map_location="cpu"))
model.eval()
print("Model loaded and set to eval mode.")

Model loaded and set to eval mode.


In [ ]:
!pip install onnx
import torch.onnx

dummy_input = torch.randn(1, 3, 32, 32)
onnx_path = f"{root}/models/model.onnx"
torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    input_names=['input'],
    output_names=['output'],
    opset_version=13,
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
)
print(f"Exported model to {onnx_path}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 90.6 MB/s eta 0:00:00
Exported model to /content/drive/MyDrive/hardware_aware_optimization/models/model.onnx


In [ ]:
size = os.path.getsize(onnx_path) / 1024 / 1024
print(f"ONNX model size: {size:.2f} MB")

ONNX model size: 8.37 MB
